# Iris with ML.net

The notebook exposes the C# API for [ML.net](https://github.com/dotnet/machinelearning) with the uses of magic command ``%%mlnet``.

In [1]:
%load_ext csharpyml

Let's first save a dataset.

In [2]:
from sklearn import datasets
import pandas
iris = datasets.load_iris()
X = iris.data
y = iris.target
df = pandas.DataFrame(
    X, columns=['Slength', 'Swidth', 'Plength', 'Pwidth'])
df["Label"] = y
df = df[["Label"] + ['Slength', 'Swidth', 'Plength', 'Pwidth']]
dest = "iris_data.txt"
df.to_csv(dest, sep=',', index=False)

In [3]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## CSharp

The following code exposes one function called ``ReturnMLClass`` which returns a simple class with two methods ``Train`` and ``Predict``. [pythonnet](http://pythonnet.github.io/) binds them to python to allow the user to call them.

In [4]:
%%mlnet ReturnMLClass

public class IrisObservation
{
    [Column("0")]
    [ColumnName("Label")]
    public string Label;

    [Column("1")]
    public float Sepal_length;

    [Column("2")]
    public float Sepal_width;

    [Column("3")]
    public float Petal_length;

    [Column("4")]
    public float Petal_width;
}

public class IrisPrediction
{
    public uint PredictedLabel;

    [VectorType(4)]
    public float[] Score;
}

public class TrainTestIris
{
    string _dataset;
    PredictionFunction<IrisObservation, IrisPrediction> _fct;

    public TrainTestIris(string iris)
    {
        _dataset = iris;
    }

    public void Train(string dest)
    {
        using (var env = new ConsoleEnvironment(verbose:false))
        {
            var args = new TextLoader.Arguments()
            {
                Separator = ",",
                HasHeader = true,
                Column = new TextLoader.Column[] {
                    new TextLoader.Column("Label", DataKind.R4, 0),
                    new TextLoader.Column("Sepal_length", DataKind.R4, 1),
                    new TextLoader.Column("Sepal_width", DataKind.R4, 2),
                    new TextLoader.Column("Petal_length", DataKind.R4, 3),
                    new TextLoader.Column("Petal_width", DataKind.R4, 4),
                }
            };

            var reader = new TextLoader(env, args);
            var concat = new ColumnConcatenatingEstimator(env,
                                                          "Features", "Sepal_length",
                                                          "Sepal_width", "Petal_length", "Petal_width");
            var km = new MulticlassLogisticRegression(env, "Label", "Features");
            var pipeline = concat.Append(km);

            IDataView trainingDataView = reader.Read(new MultiFileSource(_dataset));
            var model = pipeline.Fit(trainingDataView);

            var obs = new IrisObservation()
            {
                Sepal_length = 3.3f,
                Sepal_width = 1.6f,
                Petal_length = 0.2f,
                Petal_width = 5.1f,
            };

            _fct = model.MakePredictionFunction<IrisObservation, IrisPrediction>(env);
            using(var stdest = File.OpenWrite(dest))
                model.SaveTo(env, stdest);
        }
    }

    public IrisPrediction Predict(double sl, double sw, double pl, double pw)
    {
        var obs = new IrisObservation()
        {
            Sepal_length = (float)sl,
            Sepal_width = (float)sw,
            Petal_length = (float)pl,
            Petal_width = (float)pw,
        };
        return _fct.Predict(obs);
    }
}

public static TrainTestIris ReturnMLClass(string ds)
{ 
    return new TrainTestIris(ds);
}   

<function csharpy.runtime.compile.create_cs_function.<locals>.<lambda>(*params)>

In [5]:
obj = ReturnMLClass("iris_data.txt")

In [6]:
try:
    obj.Train("model_iris_mlnet.zip")
except Exception as e:
    print(e)

Label column 'Label' is not compatible
   à Microsoft.ML.Runtime.Training.TrainerEstimatorBase`2.CheckLabelCompatible(Column labelCol)
   à Microsoft.ML.Runtime.Training.TrainerEstimatorBase`2.CheckInputSchema(SchemaShape inputSchema)
   à Microsoft.ML.Runtime.Training.TrainerEstimatorBase`2.GetOutputSchema(SchemaShape inputSchema)
   à Microsoft.ML.Runtime.Data.EstimatorChain`1.GetOutputSchema(SchemaShape inputSchema)
   à Microsoft.ML.Runtime.Data.EstimatorChain`1.Fit(IDataView input)
   à DynamicCS.DynamicCSFunctions_ReturnMLClass.TrainTestIris.Train(String dest)


In [7]:
try:
    res = obj.Predict(4.5, 3.4, 3.4, 7.8)
    res.PredictedLabel, list(res.Score)
    print(res.PredictedLabel, list(res.Score))
except Exception as e:
    print(e)

La référence d'objet n'est pas définie à une instance d'un objet.
   à DynamicCS.DynamicCSFunctions_ReturnMLClass.TrainTestIris.Predict(Double sl, Double sw, Double pl, Double pw)


## Python with NimbusML

[nimbusml](https://docs.microsoft.com/en-us/nimbusml/overview) is an open source python binding for [ML.net](https://github.com/dotnet/machinelearning). The following example would become.

In [8]:
import numpy
df["Label"] = df.Label.astype(numpy.float32)

In [9]:
import nimbusml
from nimbusml import Pipeline
from nimbusml.linear_model import LogisticRegressionClassifier

pipe = Pipeline([LogisticRegressionClassifier()])
pipe.fit(df)

Automatically adding a MinMax normalization transform, use 'norm=Warn' or 'norm=No' to turn this behavior off.
LBFGS multi-threading will attempt to load dataset into memory. In case of out-of-memory issues, turn off multi-threading by setting trainThreads to 1.
Beginning optimization
num vars: 24
improvement criterion: Mean Improvement
L1 regularization selected 16 of 24 weights.
Not training a calibrator because it is not needed.
Elapsed time: 00:00:03.2281790


In [10]:
pred = pipe.predict_proba(df, verbose=0)

In [11]:
pred[:5]

array([[0.93744504, 0.0333313 , 0.02922361],
       [0.93744504, 0.0333313 , 0.02922361],
       [0.9379973 , 0.03308264, 0.02892021],
       [0.9368883 , 0.03358165, 0.02953005],
       [0.93744504, 0.0333313 , 0.02922361]], dtype=float32)

In [12]:
pipe.save_model("model_iris_nimbusml.zip")

## Back to ML.net

We can use this model back into [ML.net](https://github.com/dotnet/machinelearning) in C#.

In [13]:
%%mlnet ScoreIris

public class IrisObservation
{
    [Column("0")]
    [ColumnName("Label")]
    public string Label;

    [Column("1")]
    public float Sepal_length;

    [Column("2")]
    public float Sepal_width;

    [Column("3")]
    public float Petal_length;

    [Column("4")]
    public float Petal_width;
}

public class IrisPrediction
{
    public uint PredictedLabel;

    [VectorType(4)]
    public float[] Score;
}

public static IrisPrediction ScoreIris(double sl, double sw, double pl, double pw)
{
    var obs = new IrisObservation()
    {
        Sepal_length = (float)sl,
        Sepal_width = (float)sw,
        Petal_length = (float)pl,
        Petal_width = (float)pw,
    };

    var modelName = "model_iris_mlnet.zip";
    var context = new Microsoft.ML.MLContext();
    using (var st = File.OpenRead(modelName))
    {
        var model = context.Model.Load(st);
        using (var env = new ConsoleEnvironment())
        {
            var scorer = model.MakePredictionFunction<IrisObservation, IrisPrediction>(env);
            var singlePrediction = scorer.Predict(obs);
            return singlePrediction;
        }
    }
}

<function csharpy.runtime.compile.create_cs_function.<locals>.<lambda>(*params)>

In [14]:
obj2 = ScoreIris(4.5, 3.4, 3.4, 7.8)
obj2.PredictedLabel, list(obj2.Score)

(3, [8.637699e-06, 7.612758e-05, 0.9999151])